---
title: "Clarity Tables"
jupyter: "starr-oncology-data-lake-arpah"
execute:
  echo: false
---





Below the list of the EPIC Clarity tables needed to assemble the CAP forms and the pathology reports. The table name contains the link for the full public description of the table including the columns of each one and the description of the columns. 


In [ ]:
#| output: asis
#| echo: false


import requests
from lxml import html
from google.cloud import bigquery
import pandas as pd
import os
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

credentials_path = "/home/rstudio/.config/gcloud/application_default_credentials.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
project_id = 'som-rit-phi-oncology-prod'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
dataset_id = 'oncology_cap_forms_phi_irb76049_aug2025'
bq_client = bigquery.Client(project=project_id)
base_url = "https://open.epic.com/EHITables/GetTable/"
# XPath to extract text from
xpath = "/html/body/div[2]/div[2]/main/div[2]/table[2]/tbody/tr/td[2]/table/tbody/tr/td/text()"
#base_url = "https://open.epic.com/EHITables/GetTable/ABN_DOCUMENT_ID.htm"
exclude_tables = ['cap_forms', 'shc_pathology_reports']

def get_bigquery_table_names(project_id, dataset_id, exclude_tables=None):
    """
    Get all table names from a BigQuery dataset
    
    Args:
        project_id (str): The BigQuery project ID
        dataset_id (str): The BigQuery dataset ID
        exclude_tables (list): Optional list of table names to exclude
    
    Returns:
        list: List of table names in the dataset
    """
    if exclude_tables is None:
        exclude_tables = []
    
    try:
        # Get dataset reference
        dataset_ref = bq_client.dataset(dataset_id, project=project_id)
        
        # List all tables in the dataset
        tables = bq_client.list_tables(dataset_ref)
        
        # Extract table names, excluding any in the exclude list
        table_names = [
            table.table_id for table in tables 
            if table.table_id not in exclude_tables
        ]
        
        return sorted(table_names)
        
    except Exception as e:
        print(f"Error retrieving table names: {e}")
        return []

def extract_text_from_xpath(url, xpath):
    """
    Fetch HTML from URL and extract text using the specified XPath
    """
    try:
        # Send GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Parse the HTML content
        tree = html.fromstring(response.content)
        
        # Extract text using XPath
        result = tree.xpath(xpath)
        
        if result:
            # If result is a list, join the text elements
            if isinstance(result, list):
                return ' '.join(str(item).strip() for item in result if str(item).strip())
            else:
                return str(result).strip()
        else:
            return "Description not available in public EPIC documentation"
            
    except requests.RequestException as e:
        return f"Error fetching URL: {e}"
    except Exception as e:
        return f"Error parsing HTML or XPath: {e}"



table_names = get_bigquery_table_names(project_id, dataset_id, exclude_tables)

descriptions = []
urls = []
for table in table_names:
    url = f"{base_url}{table.upper()}.htm"
    extracted_text = extract_text_from_xpath(url, xpath)
    descriptions.append(extracted_text)
    urls.append(url)

df = pd.DataFrame({
    'Table Name': table_names,
    'Table Description': descriptions,
    'URL': urls
})

# Convert DataFrame to HTML and display
HTML(df.to_html(escape=False, table_id="df", index=False, render_links=True))